In [1]:
import spacy
import json
import requests

# Load spaCy model
nlp = spacy.load('en_core_web_sm')


In [6]:
API_KEY = '9f68d3dd-bd65-4d1f-bcf8-bfc8d0bc8c71'
BASE_PLAYERS_URL = 'https://api.cricapi.com/v1/players?apikey={}&search={}'
PLAYER_INFO_URL = 'https://api.cricapi.com/v1/players_info?apikey={}&id={}'

def get_players_by_name(player_name):
    try:
        response = requests.get(BASE_PLAYERS_URL.format(API_KEY, player_name))
        response.raise_for_status()
        data = response.json()
        
        if data.get("status") == "success":
            return data['data']
        else:
            print(f"API returned an error: {data}")
            return []
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return []
    except requests.exceptions.RequestException as err:
        print(f"Other error occurred: {err}")
        return []
    except json.JSONDecodeError:
        print("Error decoding the JSON response.")
        return []

def save_player_info_to_file(player_name, player_info):
    filename = f"{player_name.replace(' ', '_').lower()}_info.json"
    with open(filename, 'w') as file:
        json.dump(player_info, file, indent=4)
    print(f"Player information saved to {filename}")

def get_player_info(player_id):
    try:
        response = requests.get(PLAYER_INFO_URL.format(API_KEY, player_id))
        response.raise_for_status()
        data = response.json()
        
        if data.get("status") == "success":
            return data['data']
        else:
            print(f"API returned an error: {data}")
            return None
    except requests.exceptions.HTTPError as http_err:
        print(f"HTTP error occurred: {http_err}")
        return None
    except requests.exceptions.RequestException as err:
        print(f"Other error occurred: {err}")
        return None
    except json.JSONDecodeError:
        print("Error decoding the JSON response.")
        return None

def find_player_details(player_name):
    players = get_players_by_name(player_name)

    if len(players) == 0:
        print(f"No player found with the name '{player_name}'.")
    elif len(players) == 1:
        player = players[0]
        player_info = get_player_info(player['id'])
        if player_info:
            save_player_info_to_file(player['name'], player_info)
            return player_info
        else:
            print(f"Failed to fetch details for player with ID {player['id']}.")
    else:
        print(f"Multiple players found with the name '{player_name}'. Please provide a more specific name.")
        for player in players:
            print(f"- {player['name']} ({player['country']})")
    return None


In [7]:
def process_query(query):
    doc = nlp(query.lower())
    intent = None
    player_name = None
    
    # Use named entity recognition to find player names
    for ent in doc.ents:
        if ent.label_ == 'PERSON':
            player_name = ent.text

    # Determine intent based on keywords
    if 'info' in query or 'details' in query:
        intent = 'info'
    elif 'stats' in query or 'statistics' in query:
        intent = 'stats'
    elif 'visualize' in query or 'chart' in query:
        intent = 'visualize'
    
    return intent, player_name


In [8]:
def chat():
    print("Chatbot: Hello! How can I help you today?")
    while True:
        user_input = input("You: ")
        if user_input.lower() in ['exit', 'quit', 'bye']:
            print("Chatbot: Goodbye!")
            break
        
        intent, player_name = process_query(user_input)
        
        if intent == 'info' or (player_name and not intent):
            player_info = find_player_details(player_name)
            if player_info:
                print(f"Chatbot: Here is some information about {player_name}:")
                print(json.dumps(player_info, indent=4))
            else:
                print("Chatbot: Sorry, I couldn't find information on that player.")
        elif intent == 'stats':
            # Implement retrieving and displaying stats
            print("Chatbot: Retrieving stats... (this function is not yet implemented)")
        elif intent == 'visualize':
            # Implement visualization
            print("Chatbot: Generating visualization... (this function is not yet implemented)")
        else:
            print("Chatbot: I'm sorry, I didn't understand that. How can I assist you?")


In [ ]:
chat()

Chatbot: Hello! How can I help you today?
You: virat kohli
Player information saved to virat_kohli_info.json
Chatbot: Here is some information about virat kohli:
{
    "id": "c61d247d-7f77-452c-b495-2813a9cd0ac4",
    "name": "Virat Kohli",
    "dateOfBirth": "1988-11-05T00:00:00",
    "role": "Batsman",
    "battingStyle": "Right Handed Bat",
    "bowlingStyle": "Right-arm medium",
    "placeOfBirth": "Delhi",
    "country": "India",
    "playerImg": "https://h.cricapi.com/img/players/c61d247d-7f77-452c-b495-2813a9cd0ac4.jpg",
    "stats": [
        {
            "fn": "batting",
            "matchtype": "test",
            "stat": "m",
            "value": "113"
        },
        {
            "fn": "batting",
            "matchtype": "test",
            "stat": "inn",
            "value": "191"
        },
        {
            "fn": "batting",
            "matchtype": "test",
            "stat": "no",
            "value": "11"
        },
        {
            "fn": "batting",
     

You: dhoni
Chatbot: I'm sorry, I didn't understand that. How can I assist you?
You: ms dhoni
Chatbot: I'm sorry, I didn't understand that. How can I assist you?
You: virat
Chatbot: I'm sorry, I didn't understand that. How can I assist you?
You: virat kohli
API returned an error: {'apikey': '9f68d3dd-bd65-4d1f-bcf8-bfc8d0bc8c71', 'info': {'hitsToday': 101, 'hitsUsed': 1, 'hitsLimit': 100, 'credits': 0}, 'status': 'failure', 'reason': 'hits today exceeded hits limit'}
Failed to fetch details for player with ID c61d247d-7f77-452c-b495-2813a9cd0ac4.
Chatbot: Sorry, I couldn't find information on that player.
You: mayank agarwal
Chatbot: I'm sorry, I didn't understand that. How can I assist you?
